In [ ]:
import json
import pandas as pd
import numpy as np
import json
import warnings

### 1. Carga de los datos obtenidos.

In [ ]:
with open('resultados.json', 'r') as f:
    data= f.read()
    

In [ ]:
lista = json.loads(data)

In [ ]:
productos = []
reviews = []

for i in lista:
    for key, values in i.items():
       
        if key == 'Producto':
            productos.append(values)
        elif key == 'Reseñas':
            reviews.append(values)

        

In [ ]:
filtered_reviews = []

for review in reviews:
    if review is not None and isinstance(review, list):
        product_reviews = []
        for item in review:
            if isinstance(item, str) and len(item) == 18:
                product_reviews.append(item)
        filtered_reviews.append(product_reviews)


In [ ]:
filtered_reviews[0]

### 2. Creación del dataframe por cada usuario.

In [ ]:

max_cols = max(len(row) for row in filtered_reviews)

column_names = [f'usuario_{i+1}' if i < len(filtered_reviews[0]) else None for i in range(max_cols)]

df = pd.DataFrame(filtered_reviews, columns=column_names)
df.head()


In [ ]:
def procesar_columna(columna):
    return columna.str.split(' ', expand=True)[0]
df = df.apply(procesar_columna)


In [ ]:
df.info()

In [ ]:
df['Productos'] = productos[:692]


df = df.reindex(range(1995)).fillna(value=pd.np.nan)

In [ ]:
rating = df

### 3. Unión entre los dataframes reviews y porductos.

In [ ]:
rating['Producto']=rating['Productos']

In [ ]:
rating = rating.drop('Productos', axis=1)

In [ ]:
rating.head()

In [ ]:
productos = pd.read_csv('productos.csv')

In [ ]:
productos.head()

In [ ]:
merged= pd.merge(productos, rating, on = 'Producto')

In [ ]:
merged.head()

**3.1 Limpieza del dataframe después de aplicar el merge.**

In [ ]:
merged['Products'] = merged['Brand'] + ' - ' + merged['Name']


In [ ]:
merged.head()

In [ ]:
merged = merged.drop(columns=['Brand','Name','Producto' , 'Unnamed: 0'], axis=1)

In [ ]:
merged.head()

In [ ]:

new_order = ['Type', 'Products','Skin','usuario_1','usuario_2','usuario_3','usuario_4','usuario_5','usuario_6','usuario_7','usuario_8','usuario_9','usuario_10']
merged = merged.reindex(columns=new_order)

In [ ]:
merged.info()

Vamos a transformar los rating en numeros float, para poder trabajar con estos datos posteriormente.

In [ ]:

cols_to_convert = [col for col in merged.columns if col not in ['Type', 'Products', 'Skin']]
merged[cols_to_convert] = merged[cols_to_convert].applymap(lambda x: float(x))

merged.info()

In [ ]:
merged.usuario_4.isnull().sum()

In [ ]:
merged.loc[merged["usuario_10"].isnull()].head()

A continuación se van a eliminar aquellas filas, que tengan más de 6 valores nulos, puesto que son pocos los casos de productos con tan pocas reseñas.

In [ ]:
merged = merged.dropna(thresh=8)

In [ ]:
merged.info()

Los nulos que nos quedan se van a sustituir por la media de la fila, ya que sería la media de las reviews que recibe el producto.

In [ ]:
cols = [col for col in merged.columns if col not in ['Type', 'Products', 'Skin']]
merged['mean'] = merged[cols].mean(axis=1)[0]

In [ ]:
merged['usuario_7'].isnull().sum()

In [ ]:
merged.loc[merged["usuario_7"].isnull()].head()

In [ ]:
merged['usuario_6'] = merged['usuario_6'].fillna(merged['mean'])

In [ ]:
merged['usuario_7'] = merged['usuario_7'].fillna(merged['mean'])

In [ ]:
merged['usuario_8'] = merged['usuario_8'].fillna(merged['mean'])

In [ ]:
merged['usuario_9'] = merged['usuario_9'].fillna(merged['mean'])

In [ ]:
merged['usuario_10'] = merged['usuario_10'].fillna(merged['mean'])

In [ ]:
merged= merged.drop('mean', axis=1)

### 4. Guardamos el dataframe final.

In [ ]:
merged.to_csv('SKINCARE.csv', index=False)